In [2]:
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Flatten, Activation
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.utils import np_utils
from tensorflow.keras.optimizers import SGD
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
# Задаем seed для повторяемости результатов
numpy.random.seed(42)

# Загружаем данные
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
# Размер мини-выборки
batch_size = 32
# Количество классов изображений
nb_classes = 10
# Количество эпох для обучения
nb_epoch = 15
# Размер изображений
img_rows, img_cols = 32, 32
# Количество каналов в изображении: RGB
img_channels = 3

# Нормализуем данные
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# Преобразуем метки в категории
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Создаем последовательную модель
model = Sequential()
# Первый сверточный слой
model.add(Conv2D(32, (3, 3), padding='same',
                        input_shape=(32, 32, 3), activation='relu'))
# Второй сверточный слой
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
# Первый слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))

# Третий сверточный слой
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
# Четвертый сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))
# Второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2)))
# Слой регуляризации Dropout
model.add(Dropout(0.25))
# Слой преобразования данных из 2D представления в плоское
model.add(Flatten())
# Полносвязный слой для классификации
model.add(Dense(512, activation='relu'))
# Слой регуляризации Dropout
model.add(Dropout(0.5))
# Выходной полносвязный слой
model.add(Dense(nb_classes, activation='softmax'))

# Задаем параметры оптимизации
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])
# Обучаем модель
model.fit(X_train, Y_train,
              batch_size=batch_size,
              epochs=nb_epoch,
              validation_split=0.1,
              shuffle=True,
              verbose=2)

# Оцениваем качество обучения модели на тестовых данных
scores = model.evaluate(X_test, Y_test, verbose=0)
print("Точность работы на тестовых данных: %.2f%%" % (scores[1]*100))

170508288/170498071 [==============================] - 349s 2us/step


h:\pythonproject\lab\venv\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Epoch 1/15
1407/1407 - 57s - loss: 1.7532 - accuracy: 0.3523 - val_loss: 1.4649 - val_accuracy: 0.4730 - 57s/epoch - 41ms/step
Epoch 2/15
1407/1407 - 57s - loss: 1.3345 - accuracy: 0.5177 - val_loss: 1.1185 - val_accuracy: 0.6024 - 57s/epoch - 40ms/step
Epoch 3/15
1407/1407 - 57s - loss: 1.1566 - accuracy: 0.5841 - val_loss: 1.0120 - val_accuracy: 0.6512 - 57s/epoch - 40ms/step
Epoch 4/15
1407/1407 - 57s - loss: 1.0448 - accuracy: 0.6283 - val_loss: 0.9751 - val_accuracy: 0.6530 - 57s/epoch - 40ms/step
Epoch 5/15
1407/1407 - 57s - loss: 0.9589 - accuracy: 0.6585 - val_loss: 0.8423 - val_accuracy: 0.7174 - 57s/epoch - 40ms/step
Epoch 6/15
1407/1407 - 57s - loss: 0.8887 - accuracy: 0.6865 - val_loss: 0.8592 - val_accuracy: 0.7056 - 57s/epoch - 40ms/step
Epoch 7/15
1407/1407 - 57s - loss: 0.8383 - accuracy: 0.7060 - val_loss: 0.7338 - val_accuracy: 0.7500 - 57s/epoch - 40ms/step
Epoch 8/15
1407/1407 - 58s - loss: 0.7929 - accuracy: 0.7249 - val_loss: 0.7816 - val_accuracy: 0.7378 - 58s/ep

In [6]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("cifar10_model.json", "w")
json_file.write(model_json)
json_file.close()
model.save_weights("cifar10_model.h5")

print ("Сохранили Model")

Сохранили Model


In [2]:
from keras.models import model_from_json

json_file = open("cifar10_model.json", "r")
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("cifar10_model.h5")
loaded_model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
print ("Загрузили Model")

# ***** Загружаем изображение в Keras:
import numpy as np
from keras.preprocessing import image

img_path = 'sportcar.jpg'
img = image.load_img(img_path, target_size=(32, 32))
# В отличие от рукописных цифр, в этот раз изображение цветное и его размер 32х32, в соответствии с форматом CIFAR-10. Преобразуем картинку в массив numpy:

x = image.img_to_array(img)
x /= 255
x = np.expand_dims(x, axis=0)

# Запускаем распознавание объекта:

prediction = loaded_model.predict(x)
# Для удобства вывода задаем список с названиями классов объектов:

classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']
# Печатаем результат распознавания:

print(classes[np.argmax(prediction)])

Загрузили Model
автомобиль
